#### 1) Прочитать статьи про BPR, WARP loss

#### 2) Сделать грид серч текущей модели, смотрите на метрику precision@5, считаем на тесте нашей функцией
    Подбор параметров:
    - Loss
    - no_components
    - regularization

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [40]:
from lightfm.evaluation import precision_at_k, recall_at_k

from metrics import precision_at_k as custom_precision, recall_at_k
from utils import prefilter_items

In [41]:
data = pd.read_csv('retail_train.csv')

item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [42]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [43]:
n_items_before = data_train['item_id'].nunique()

data_train_filtered = prefilter_items(data_train, take_n_popular=5000, item_features=item_features)

n_items_after = data_train_filtered['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

/content/utils.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 86865 to 5001


In [44]:
user_item_matrix = pd.pivot_table(data_train_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) 


sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,117847,818981,819255,819308,819400,819487,819590,819594,819840,819845,...,15926775,15926844,15926886,15972074,15972298,15972565,15972790,16100266,16729299,16729415
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

test_user_item_matrix = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [47]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [48]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
117847,450.0,NUTRITION,National,REFRIGERATED,SOY/RICE MILK,64 OZ
818981,194.0,GROCERY,National,COLD CEREAL,ALL FAMILY CEREAL,10.4 OZ


In [50]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())
users_ids_row = data_train_filtered['user_id'].apply(lambda x: userid_to_id[x]).values.astype(int)
items_ids_row = data_train_filtered['item_id'].apply(lambda x: itemid_to_id[x]).values.astype(int)

In [56]:
def model_light (no_components=40, loss='bpr', learning_rate=0.01, item_alpha=0.4, user_alpha=0.1):
    model = LightFM(no_components=no_components,
                    loss=loss, # "logistic","bpr", "warp"
                    learning_rate=learning_rate,
                    item_alpha=item_alpha,
                    user_alpha=user_alpha,
                    random_state=42,
                    k=5,
                    n=15,
                    max_sampled=100)
    return model

def fit_model(model):
    return model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
              sample_weight=coo_matrix(user_item_matrix),
              user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
              item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
              epochs=20,
              num_threads=20,
              verbose=True)

def predict_model(model):
    predictions = model.predict(user_ids=users_ids_row,
                                item_ids=items_ids_row,
                                user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                num_threads=10)
    return predictions

def predict_results_df (predictions, df_result_for_metrics):
    data_train_filtered['score'] = predictions

    predict_result = data_train_filtered[['user_id','item_id','score']][data_train_filtered.item_id != 999999].drop_duplicates().sort_values(by=['user_id','score'], ascending=False).groupby('user_id')['item_id'].unique().reset_index()

    df_result_for_metrics = result.merge(predict_result, on='user_id', how='inner')

    return df_result_for_metrics

In [55]:
def grid_search(no_components=None, loss=None, learning_rate=None, item_alpha=None, user_alpha=None):
    if loss is None:
        loss = ['brp']
    if no_components is None:
        no_components = [40]
    if learning_rate is None:
        learning_rate = [0.01]
    if item_alpha is None:
        item_alpha = [0.4]
    if user_alpha is None:
        user_alpha = [0.1]
    df_result_for_metrics = result
    predict_results = pd.DataFrame()
    for no_components_ in no_components:
      for loss_ in loss:
        for learning_rate_ in learning_rate:
          for item_alpha_ in item_alpha:
            for user_alpha_ in user_alpha:

                model = model_light(learning_rate=learning_rate_, item_alpha=item_alpha_, user_alpha=user_alpha_)
                fit_model(model)
                predictions = predict_model(model)

                df_result_for_metrics = predict_results_df(predictions, df_result_for_metrics)

                precision = round(df_result_for_metrics.apply(lambda row: custom_precision(row['item_id'], row['actual'],k=5), axis=1).mean(), 4)

                name = f'no_components: {no_components_} loss: {loss_} learning_rate: {learning_rate_} item_alpha: {item_alpha_} user_alpha: {user_alpha_}'
                df_result_for_metrics = df_result_for_metrics.rename(columns={'item_id': name})

                predict_results = predict_results.append(pd.DataFrame([precision], columns=['precision'], index=[name]))

                print(f"Precision: {precision} | {name}")

    predict_results = predict_results.sort_values(by=['precision'], ascending=False)
    best = f'Лучшие параметры: {predict_results.head(1).precision.tolist()[0]}, {predict_results.head(1).index.tolist()[0]}'
    print(f'\n{best}')

    return df_result_for_metrics, predict_results, best

In [57]:
results, predict_results, best = grid_search(no_components = [40, 60],
                          loss = ['bpr', 'warp'],                       
                          learning_rate = [0.01, 0.05],
                          item_alpha = [0.4, 0.3],
                          user_alpha = [0.1, 0.2])

Epoch: 100%|██████████| 20/20 [00:40<00:00,  2.03s/it]


Precision: 0.1422 | no_components: 40 loss: bpr learning_rate: 0.01 item_alpha: 0.4 user_alpha: 0.1


Epoch: 100%|██████████| 20/20 [00:40<00:00,  2.02s/it]


Precision: 0.1351 | no_components: 40 loss: bpr learning_rate: 0.01 item_alpha: 0.4 user_alpha: 0.2


Epoch: 100%|██████████| 20/20 [00:40<00:00,  2.05s/it]


...

Precision: 0.1083 | no_components: 60 loss: warp learning_rate: 0.05 item_alpha: 0.4 user_alpha: 0.2


Epoch: 100%|██████████| 20/20 [00:40<00:00,  2.04s/it]


Precision: 0.1297 | no_components: 60 loss: warp learning_rate: 0.05 item_alpha: 0.3 user_alpha: 0.1


Epoch: 100%|██████████| 20/20 [00:40<00:00,  2.01s/it]


Precision: 0.0549 | no_components: 60 loss: warp learning_rate: 0.05 item_alpha: 0.3 user_alpha: 0.2

Лучшие параметры: 0.1429, no_components: 40 loss: warp learning_rate: 0.01 item_alpha: 0.3 user_alpha: 0.1
